# Import the modules 

In [1]:
# import the general modules
from optlang.symbolics import Zero
import cobra
import cplex
import sys

# import kbase
import os
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'KCYWCM5Q3ZFAGQKDG3ESMJXGHQPK7UMN'
kbase = cobrakbase.KBaseAPI(token)

# define the example individual model and associated API media package
#model = kbase.get_from_ws('e_coli_core', 94253)
model = kbase.get_from_ws('e_coli_core.kb', 94253)
#model = kbase.get_from_ws('E_iAH991V2', 94253)
model.solver = 'optlang-cplex'

# local ModelSEED database path
modelseed_db_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'

cobrakbase 0.2.8


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# RevBin package test

The RevBin package prevents directional fluxes through reactions that have net zero flux. This is exemplified in the following demonstration, where the forward progression of the reversible ethanol transport reaction (rxn08427_c0) is optimized despite lacking the ability to have net flux. The unconstrained simulation possesses a maximal objective value, while the RevBin-constrained simulation possesses an objective value of zero.    

## Unconstrained simulation

In [2]:
model = kbase.get_from_ws('e_coli_core.kb', 94253)

with model:
    # set the objective function
    model.objective = model.reactions.rxn08427_c0.forward_variable 
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## RevBin-constrained simulation

### Constrain and execute the model

In [3]:
from modelseedpy.fbapkg import revbinpkg

# apply the RevBin constraints 
revbin = revbinpkg.RevBinPkg(model)
revbin.build_package()

with model:
    # set the objective function
    model.objective = model.reactions.rxn08427_c0.forward_variable 
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_e0,EX_cpd00007_e0,100,0,0.00%
cpd00020_e0,EX_cpd00020_e0,800,3,54.55%
cpd00363_e0,EX_cpd00363_e0,1000,2,45.45%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00001_e0,EX_cpd00001_e0,-200,0,0.00%
cpd00071_e0,EX_cpd00071_e0,-1000,2,45.45%
cpd00221_e0,EX_cpd00221_e0,-800,3,54.55%


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# SimpleThermo package test

The SimpleThermo package prevents flux through thermodynamically infeasible reactions. This is exemplified in the following demonstrations. 

The first demonstration executes the model in an empty media, which is an infeasible environment for growth. The unconstrained model grows, while the SimpleThermo-constrained model does not grow. 
The second demonstration optimizes the model for an internal loop (FRD7_c0) in the *E. coli* core model. The unconstrained model possesses a maximal objective value, while the SimpleThermo-constrained model possesses an objective value of zero.   

## Unconstrained simulation

### Execute the model with conventional objective

In [4]:
model = kbase.get_from_ws('e_coli_core.kb', 94253)

with model:
    # edit the model media
    medium = model.medium
    medium = {}
    model.medium = medium
    display(model.medium)

    # execute the model
    display(model.summary())

{}

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### Execute the model with internal loop objective

In [5]:
with model:
    model.objective = model.reactions.FRD7_c0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


## SimpleThermo-constrained simulation

### Constrain the model

In [6]:
# ensure that all charges are integers
model = kbase.get_from_ws('e_coli_core.kb', 94253)
for metabolite in model.metabolites:
    try:
        metabolite.charge = int(metabolite.charge)
    except:
        print(metabolite, metabolite.charge)
        metabolite.charge = None
        
# apply the simple thermodynamic constraints 
from modelseedpy.fbapkg import simplethermopkg
stp = simplethermopkg.SimpleThermoPkg(model)
stp.build_package(stp.parameters)

# export the constrained LP file 
with open('SimpleThermo.lp', 'w') as out:
    out.write(str(model.solver))

### Execute the model in empty media

In [7]:
with model:
    # edit the model media
    medium = model.medium
    medium = {}
    model.medium = medium
    display(model.medium)

    # execute the model
    display(model.summary())

{}

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### Execute the model with an optimized internal flux

In [8]:
with model:
    # execute the simulation
    model.objective = model.reactions.FRD7_c0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Brainstorming

In [9]:
for reaction in model.reactions:
    print(reaction.id)

rxn00549_c0
rxn05312_c0
rxn00247_c0
rxn00786_c0
rxn08865_c0
rxn01200_c0
rxn08525_c0
FRD7_c0
rxn00545_c0
rxn09270_c0
rxn00173_c0
rxn08288_c0
rxn00459_c0
rxn00083_c0
rxn00097_c0
rxn32421_c0
rxn00154_c0
rxn19316_c0
rxn00187_c0
rxn00558_c0
rxn05319_c0
rxn00747_c0
rxn05468_c0
rxn35242_c0
rxn08535_c0
rxn01115_c0
rxn00225_c0
rxn00500_c0
rxn08350_c0
rxn20162_c0
rxn01333_c0
rxn30674_c0
rxn01100_c0
rxn39085_c0
rxn08173_c0
rxn00285_c0
rxn00161_c0
rxn00543_c0
rxn05297_c0
rxn05467_c0
rxn05488_c0
rxn00085_c0
rxn00062_c0
rxn34258_c0
rxn00171_c0
rxn05559_c0
rxn09272_c0
rxn00604_c0
rxn00159_c0
rxn08972_c0
rxn08542_c0
rxn01106_c0
rxn00198_c0
rxn01476_c0
rxn00781_c0
rxn09295_c0
rxn00785_c0
rxn08032_c0
rxn00189_c0
rxn00336_c0
rxn00147_c0
rxn09269_c0
rxn00248_c0
rxn00974_c0
rxn05493_c0
rxn00777_c0
rxn08094_c0
rxn05226_c0
rxn05155_c0
rxn08427_c0
rxn00184_c0
rxn05469_c0
rxn32587_c0
rxn01116_c0
bio1
EX_cpd00013_e0
EX_cpd00082_e0
EX_cpd00023_e0
EX_cpd00011_e0
EX_cpd00053_e0
EX_cpd00363_e0
EX_cpd00009_e0
EX_cpd

In [10]:
objective_coefficient = {}
objective_coefficient[model.reactions.rxn08427_c0.forward_variable] = 1

# define the maximum progression
model.objective = model.problem.Objective(Zero, direction='max')
model.objective.set_linear_coefficients(objective_coefficient)

# execute the model
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_e0,EX_cpd00007_e0,100,0,0.00%
cpd00020_e0,EX_cpd00020_e0,800,3,54.55%
cpd00363_e0,EX_cpd00363_e0,1000,2,45.45%
Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00001_e0,EX_cpd00001_e0,-200,0,0.00%
cpd00071_e0,EX_cpd00071_e0,-1000,2,45.45%
cpd00221_e0,EX_cpd00221_e0,-800,3,54.55%


In [11]:
forv = model.reactions.rxn08427_c0.forward_variable 
print(forv)
dir(model.reactions.rxn08427_c0)

0 <= rxn08427_c0 <= 1000.0


['__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_associate_gene',
 '_check_bounds',
 '_dissociate_gene',
 '_gene_reaction_rule',
 '_genes',
 '_get_rev_and_max_min_flux',
 '_id',
 '_lower_bound',
 '_metabolites',
 '_model',
 '_repr_html_',
 '_set_id_with_model',
 '_to_json',
 '_update_awareness',
 '_upper_bound',
 'add_metabolites',
 'aliases',
 'annotation',
 'boundary',
 'bounds',
 'build_reaction_from_string',
 'build_reaction_string',
 'check_mass_balance',
 'compartment',
 'compartments',
 'copy',
 